In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

# For typing
from typing import Dict
from pandas import DataFrame

# Import files to dataframes
data_path = Path("../data/processed/")
csv_files = [file for file in os.listdir(data_path) if file.endswith(".csv")]
dfs: Dict[str, DataFrame] = {}

for csv in csv_files:
    with open(data_path.joinpath(csv), 'r') as f:
        if '|' in f.readlines(1)[0]:
            sep = '|'
        else:
            sep = ','
        df = pd.read_csv(data_path.joinpath(csv), engine="python", encoding='utf-8', sep=sep)
        dfs[csv]= df
    

In [110]:
# EDA on victimasmuestra_transformed
victimas = dfs[list(dfs.keys())[0]]
pd.set_option('display.max_columns', None)
victimas.head(), victimas.info(), victimas.nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 36 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   FUENTE                      499 non-null    object
 1   IDPERSONA                   499 non-null    int64 
 2   IDHOGAR                     499 non-null    int64 
 3   TIPODOCUMENTO               499 non-null    object
 4   DOCUMENTO                   499 non-null    int64 
 5   PRIMERNOMBRE                499 non-null    object
 6   SEGUNDONOMBRE               370 non-null    object
 7   PRIMERAPELLIDO              499 non-null    object
 8   SEGUNDOAPELLIDO             474 non-null    object
 9   FECHANACIMIENTO             499 non-null    object
 10  PERTENENCIAETNICA           499 non-null    object
 11  GENERO                      499 non-null    object
 12  HECHO                       499 non-null    object
 13  FECHAOCURRENCIA             499 non-null    object

(         FUENTE  IDPERSONA  IDHOGAR TIPODOCUMENTO   DOCUMENTO PRIMERNOMBRE  \
 0  RUV LEY 1448   10033866  2008207            TI  1015192770      MERALDO   
 1  RUV LEY 1448   10090212  2027620            CC    43275207         OLGA   
 2  RUV LEY 1448   10050025  2016284            CC  1045490491       VICTOR   
 3  RUV LEY 1448   10036288  2012386            TI  1122677390        KAROL   
 4  RUV LEY 1448   10090261  2027631            TI  1058078879     MAURICIO   
 
   SEGUNDONOMBRE PRIMERAPELLIDO SEGUNDOAPELLIDO FECHANACIMIENTO  \
 0           NaN          CAMPO       QUERAGAME      30/04/2007   
 1      PATRICIA          VICHE         NAVARRO      05/01/1980   
 2           NaN     BLANQUICET           PEREZ      22/09/1986   
 3       DAYANNA         PULIDO            LASO      27/02/2007   
 4           NaN        ACEVEDO       JARAMILLO      15/12/2008   
 
               PERTENENCIAETNICA  GENERO  \
 0                      INDIGENA  HOMBRE   
 1                       NINGUNA

In [ ]:
# Column name filtering
unique_25 =  victimas.nunique()[(victimas.nunique() <= 25)]
to_categorical = unique_25.index
to_date = pd.Index([date for date in victimas.columns if 'FECHA' in date], )

# Data type casting
## Dates
for date_col in to_date:
   victimas[date_col] = pd.to_datetime(victimas[date_col], format="%d/%m/%Y")

## Categorical
for col in to_categorical:
    if victimas[col].dtype == 'O' and (victimas[col].str.len() > 30).any():
        continue
    
    if victimas[col].dtype == 'O' and victimas[col].str.contains('/').any():
        victimas[col] = victimas[col].str.split('/', regex=False).explode(ignore_index=True).astype('category')

    victimas[col] = victimas[col].astype('category')
    
print("Datetime: ", victimas.select_dtypes( "datetime64[ns]").columns, "\n\nCategorical: ", victimas.select_dtypes("category").columns)


Datetime:  Index(['FECHANACIMIENTO', 'FECHAOCURRENCIA', 'FECHAREPORTE',
       'FECHAVALORACION'],
      dtype='object') 

Categorical:  Index(['FUENTE', 'TIPODOCUMENTO', 'PERTENENCIAETNICA', 'GENERO', 'HECHO',
       'TIPOPOBLACION', 'TIPOVICTIMA', 'ESTADOVICTIMA', 'TIPODESPLAZAMIENTO',
       'RELACION', 'CODIGOHECHO', 'DISCAPACIDAD'],
      dtype='object')
